In [1]:
import findspark

findspark.init()
findspark.find()

'C:\\Spark\\spark-3.1.2-bin-hadoop3.2'

In [2]:
import sparknlp

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql.types import *

spark_conf = SparkConf()
spark_conf = spark_conf.setAppName('spark nlp test')\
    .setAppName('master[*]')\
    .set("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

In [3]:
import os

path = os.path.join('../Data', '3구간', '1시기', '1시기_ST')

texts = spark.sparkContext.wholeTextFiles(path)

schema = StructType([
    StructField('filename', StringType()),
    StructField('text', StringType()),
])

texts_df = spark.createDataFrame(texts, schema)

In [4]:
texts_df = texts_df.withColumn(
    'article_name',
    fun.split('filename', '/').getItem(7)
)

texts_df.limit(5).toPandas()

,filename,text,article_name
0,file:/c:/Text-Mining-Project/Data/3구간/1시기/1시기_...,America's Germany\r\n\r\nAmid ruins the occupa...,America_s Germany completed.txt
1,file:/c:/Text-Mining-Project/Data/3구간/1시기/1시기_...,ILYA EHRENBURG'S AMERICA\r\n\r\nTranslations o...,Ilyas America completed.txt
2,file:/c:/Text-Mining-Project/Data/3구간/1시기/1시기_...,NO REST FOR THE\r\nWEARY RUSSIANS\r\n\r\nJOHN ...,No Rest for Weary Russians completed.txt
3,file:/c:/Text-Mining-Project/Data/3구간/1시기/1시기_...,"Our Ally, Russia\r\n\r\nON THURSDAY, September...",Our ally_1946_5_6_7.txt
4,file:/c:/Text-Mining-Project/Data/3구간/1시기/1시기_...,Our record in Japan \r\n\r\nMaxwell Stewart \r...,Our record in Japan completed.txt


### expalin_document_ml

-> transformer(변환자) 1개, annotator(애너테이터) 6개가 존재
-> annotator : 특정 nlp 기술을 표형하는 단어

1. Document Assembler : 문서를 포함하는 열을 생성하는 변환자
2. Sentense Segmenter : 문서의 문장을 생성하는 애너테이터
3. Tokenizer : 문장의 토큰을 생성하는 애터테이터
4. SpellChecker : 맞춤법이 교정된 토큰을 생성하는 애너테이터
5. Stemmer : 토큰의 어간을 생성
6. Lemmatizer : 토큰의 표제어를 생성
7. POS Tagger : 연관된 토큰의 품사를 생성

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline('explain_document_ml', lang='en')

In [ ]:
# Example
pipeline.annotate('Hellu wrold!')

In [ ]:
procd_texts_df = pipeline.annotate(texts_df, 'text')

procd_texts_df.printSchema()

In [ ]:
procd_texts_df.show(n=2, truncate=100, vertical=True)

In [ ]:
from sparknlp import Finisher

finisher = Finisher()
finisher = finisher

# Lemma(표제어) 열 가져오기
finisher = finisher.setInputCols(['lemmas'])
# 단일 공백으로 표제어 분리하기
finisher = finisher.setAnnotationSplitSymbol(' ')

finished_texts_df = finisher.transform(procd_texts_df)

finished_texts_df.show(n=1, truncate=100, vertical=True)

In [ ]:
finished_texts_df.select('finished_lemmas').take(1)